In [14]:
import os
import glob
import tqdm
import torch
import argparse
from scipy.io.wavfile import write

from model.generator import Generator
from utils.hparams import HParam, load_hparam_str
from utils.stft import TacotronSTFT
from utils.utils import read_wav_np

MAX_WAV_VALUE = 32768.0

In [9]:
checkpoint = torch.load("./assets/model.pt")
hp = hp = HParam("./config/default.yaml")

In [21]:
stft = TacotronSTFT(filter_length=hp.audio.filter_length,
                    hop_length=hp.audio.hop_length,
                    win_length=hp.audio.win_length,
                    n_mel_channels=hp.audio.n_mel_channels,
                    sampling_rate=hp.audio.sampling_rate,
                    mel_fmin=hp.audio.mel_fmin,
                    mel_fmax=hp.audio.mel_fmax)

sr, wav = read_wav_np("./assets/LJ001-0005.wav")
wav = torch.from_numpy(wav).unsqueeze(0)
mel = stft.mel_spectrogram(wav)

In [24]:
model = Generator(hp.audio.n_mel_channels).cuda()
model.load_state_dict(checkpoint['model_g'])
model.eval(inference=False)
mel = mel.cuda()
audio = model.inference(mel)
audio = audio.cpu().detach().numpy()

RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 1.96 GiB total capacity; 665.83 MiB already allocated; 15.12 MiB free; 64.17 MiB cached)

In [1]:
model = Generator(hp.audio.n_mel_channels).cuda()
model.load_state_dict(checkpoint['model_g'])
model.eval(inference=False)

with torch.no_grad():
    for melpath in tqdm.tqdm(glob.glob(os.path.join(args.input_folder, '*.mel'))):
        mel = torch.load(melpath)
        if len(mel.shape) == 2:
            mel = mel.unsqueeze(0)
        mel = mel.cuda()

        audio = model.inference(mel)
        audio = audio.cpu().detach().numpy()

        out_path = melpath.replace('.mel', '_reconstructed_epoch%04d.wav' % checkpoint['epoch'])
        write(out_path, hp.audio.sampling_rate, audio)

Downloading: "https://github.com/seungwonpark/melgan/archive/master.zip" to /home/sedrasmith/.cache/torch/hub/master.zip


URLError: <urlopen error Remote end closed connection without response>